# Harmony Regridding Service

### Contact:

* **Slack:** #harmony_service_providers
* **Jira:** [SDPS Data Services](https://bugs.earthdata.nasa.gov/secure/RapidBoard.jspa?rapidView=757&view=planning.nodetail)

## What is the Harmony Regridding Service?

The Harmony Regridding Services is a [Harmony](https://harmony.earthdata.nasa.gov) backend service that transforms the data within a specified input granule to a single geographic grid. The initial version of this service is only compatible with geographic source and target grids. The transformed output is returned to the user in the form of a standard Harmony STAC object, which contains links to the output granule in an AWS S3 bucket.

The Harmony Regridding Service leverages the `earthdata-varinfo` Python package (see: [here](https://github.com/nasa/earthdata-varinfo)) to map variable dependencies and group variables by horizontal spatial dimensions. `pyresample` (see [here](https://pyresample.readthedocs.io/en/latest/)) is used to perform the resampling in a horizontal spatial plane.


## Capabilities

* Transforming a granule with data in one or more geographic grids to another, single target geographic grid.
* Can handle N-dimensional input variables (e.g., those with only horizontal spatial dimesions, or also those with temporal and/or vertical spatial dimensions, etc).
* Preservation of file variable hierarchy.
* NetCDF-4 output.


## Data requirements

* L3 or L4 data, on one or more geographic grid(s).
* NetCDF-4 file format.
* A UMM-S record associated with the UMM-C record(s) for applicable collection(s).

## How the Harmony Regridding Service processes a request

* The Harmony Regridding Service receives a Harmony message, including `scaleExtent` and either `scaleSize` or `height` and `width`.
* The service validates that sufficient information exists to define the target grid, and that the grid is self-consistent (e.g., `scaleSize.x = (scaleExtent.x.max - scaleExtent.x.min) / width`).
* A `pyresample` [AreaDefinition](https://pyresample.readthedocs.io/en/latest/geo_def.html#areadefinition) is created for the horizontal spatial dimensions of the target grid.
* The service identifies all combinations of horizontal spatial dimensions used for gridded dimensions. A `pyresample` [DaskEWAResampler](https://pyresample.readthedocs.io/en/latest/swath.html#resampler) is created for each pair of horizontal spatial dimensions.
* The weights from the input grids to the target grid are calculated for each `DaskEWAResampler`, using the Elliptical Weighted Averaging methodology. (For non-elliptical grids, the ellipses are circular)
* The weights for each `DaskEWAResampler` object are applied to each applicable variable, and the output variables are written to an output file. This output file will also contain new 1-D dimension variables, describing the horizontal spatial dimensions of the target grid, along with the target CRS.

## Examples

The following examples use `harmony-py` (see: [here](https://github.com/nasa/harmony-py)) to make requests to Harmony. `harmony-py` is distributed via the Python Package Index (PyPI), and can be installed to your current Python environment via Pip:

```bash
$ pip install harmony-py
```

Other Python requirements are listed in `docs/pip_requirements.txt`. It is recommended to install all required packages via the following commands:

```bash
$ conda create --name regridder-demo python=3.11 --channel conda-forge --channel defaults -y
$ conda activate regridder-demo
$ pip install -r pip_requirements.txt
```

### Sample collection

The requests below process data from the MERRA-2 hourly time-averaged collection ([M2T1NXSLV](https://cmr.earthdata.nasa.gov/search/concepts/C1276812863-GES_DISC.html)), containing air temperature, wind components, pressure and precipitation related variables.

The figure below shows the input data, with a single time slice for the specific humidity at 850 hPa. The input grid has a longitudinal resolution of 0.625 degrees east, and a latitudinal resolution of 0.5 degrees north.

![](M2T1NXSLV_2021-06-01_specific_humidity_input.png)

### Initial setup

The following cell imports required packages and establishes a Harmony client to make requests to the UAT environment of Harmony.

In [ ]:
import matplotlib.pyplot as plt
import panel.widgets as pnw
import xarray as xr
from harmony import Client, Collection, Environment, Request

harmony_client = Client(env=Environment.UAT)

# Collection short name can also be used in place of the concept ID to define a
# `harmony.Collection` object:
merra_collection = Collection(id='C1245662776-EEDTEST')
merra_granules = ['G1245662789-EEDTEST', 'G1245662790-EEDTEST']

demo_directory = '.'

## Regridding via the `grid` parameter:

The `grid` parameter to the Harmony API is used to identify a [UMM-Grid object](https://github.com/nasa/Common-Metadata-Repository/tree/master/schemas/resources/schemas/grid) in CMR. Harmony will extract information from the grid object with a matching name and translate the information to standard Harmony message parameters:

* `scaleSize`
* `scaleExtent`
* `height` and `width`
* `crs`

The request below will use a grid object that is geographic with whole-Earth coverage. Each grid cell has a horizontal spatial resolution of 1 degree in both the longitudinal and latitudinal directions.

The output will be downloaded locally, and the file name will have a `_regridded` suffix, which is part of the standard Harmony syntax to indicate the transformation that has been performed on the input granule.

In [ ]:
grid_request = Request(
    collection=merra_collection, granule_id=merra_granules[0], grid='GEOS1x1test'
)

grid_job_id = harmony_client.submit(grid_request)
harmony_client.wait_for_processing(grid_job_id, show_progress=True)

downloaded_grid_output = [
    file_future.result()
    for file_future in harmony_client.download_all(
        grid_job_id, overwrite=True, directory=demo_directory
    )
]
print(f'Downloaded: {", ".join(downloaded_grid_output)}')

### Plotting the output:

The following two cells will firstly show the `xarray` data structure for this granule.

The first cell shows the standard `xarray` tabular output for the granule. Some interesting things to notice are:

* The listed dimensions at the top of the dataset show the number of pixels in each dimension. As each pixel has a resolution of 1 degree, a whole-Earth coverage will result in 360 longitude elements and 180 latitude elements.
* The values for the longitude and latitude dimensions can be seen. Each cell is centred at a half degree, as expected.
* The data variables include a CF-Convention compliant "crs" variable, containing metadata attributes for the grid.
* All other data variables use the new time, lat and lon variables, and so are mapped to the target grid.

The second cell shows the specific humidity at 850 hPa variable, as shown in the image of the input. The `hvplot` interactive widget can used to automatically iterate through all time slices, showing how the specific humidity changed throughout the day.

In [ ]:
ds_grid = xr.open_dataset(downloaded_grid_output[0])
ds_grid

In [ ]:
player_grid = pnw.Player(
    name='time', start=0, end=23, loop_policy='once', interval=200, width=900
)

ax = (
    ds_grid.Q850.interactive(loc='bottom', width=900, height=600)
    .isel(time=player_grid)
    .plot(
        cmap=plt.cm.turbo,
        vmin=0.0,
        vmax=0.02,
        cbar_kwargs={'format': '{x:.3}', 'fraction': 0.0235},
        xlim=[-180, 180],
        ylim=[-90, 90],
    )
)
ax.axes.set_aspect('equal')

In [ ]:
ds_grid.close()

## Regridding with explicit grid definitions:

The same parameters that Harmony extracts from a UMM-Grid object can be explicitly defined in a Harmony request to achieve the same output. However, if a UMM-Grid object exists, that is the simpler and preferred method to call the service. An end-user can use this alternative method to specify a custom grid if a corresponding UMM-Grid object does not exist.

The request below defines the same grid as before, but uses a different granule to avoid naming conflicts in downloaded output:

In [ ]:
detailed_request = Request(
    collection=merra_collection,
    granule_id=merra_granules[1],
    scale_size=(1.0, 1.0),
    scale_extent=(-180, -90, 180, 90),
    crs='EPSG:4326',
    height=180,
    width=360,
)

detailed_job_id = harmony_client.submit(detailed_request)
harmony_client.wait_for_processing(detailed_job_id, show_progress=True)

downloaded_detailed_output = [
    file_future.result()
    for file_future in harmony_client.download_all(
        detailed_job_id, overwrite=True, directory=demo_directory
    )
]
print(f'Downloaded: {", ".join(downloaded_detailed_output)}')

### Detailed request output:

The information below will show that the output, as specified by the `scaleExtent`, `scaleSize`, `height`, `width` and `crs`, has the same characteristic as the request specifying the UMM-Grid object: a whole-Earth grid with 1 degree horizontal spatial resolution.

In [ ]:
ds_detailed = xr.open_dataset(downloaded_detailed_output[0])
ds_detailed

In [ ]:
ds_detailed.close()